# Powerpoint Slides Summarizer

This converts a Power Point presentation into notes that a student can easily skim through.

Concepts Used:
- Converting Contents of PPT to text via python-pptx
- User and System Prompts
- Use of Open AI GPT-4o-mini via API key


In [1]:
!pip install python-pptx

In [2]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
# creating an instance
openai = OpenAI()

In [18]:
from pptx import Presentation

class PowerPoint():
    def __init__(self,ppt):
        """
        Creates a PowerPoint object, with name and text.
        """
        self.ppt = ppt
        self.title = os.path.basename(ppt)
        self.title = self.title[:-5]
        self.text = self.extract_text()

    def extract_text(self):
        """
        Extracts text from powerpoint.
        """
        prs = Presentation(self.ppt)
        text_content = []
    
        for slide in prs.slides:
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    text_content.append(shape.text)
    
        return "\n".join(text_content)
    

In [6]:
system_prompt = "You are an assistant that analyzes the contents \
of a PowerPoint presentation, and provides a summary in the style of \
a cheat-sheet, for students to easily learn key concepts from.\
You are to ignore text that might be navigation-related\
and respond in Markdown."

In [7]:
# A function that writes a User Prompt that asks for summaries of PowerPoints:

def user_prompt_for(powerpoint):
    user_prompt = f"You are looking at a website titled {powerpoint.title}"
    user_prompt += "\nThe contents of this powerpoint are as follows; \
please provide a summary of the content in markdown. \
If it includes a question bank, add that along with short answers too.\n\n"
    user_prompt += powerpoint.text
    return user_prompt

In [8]:
def messages_for(powerpoint):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(powerpoint)}
    ]

In [9]:
# And now: call the OpenAI API. 

def summarize(powerpoint_path):
    powerpoint = PowerPoint(powerpoint_path)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(powerpoint)
    )
    return response.choices[0].message.content, powerpoint.title

In [19]:
def display_summary(url, save_to_file=True):
    summary, title = summarize(url)
    if save_to_file:
        with open(f'{title}.txt', 'w') as file:
            file.write(summary)
    display(Markdown(summary))

In [20]:
ppt_file = "Theoretical Perspectives on Media and Technology.pptx" 
display_summary(ppt_file)

# Cheat-Sheet: Theoretical Perspectives on Media and Technology

## Introduction to Media and Technology
- **Impact of Technology**: Transforms communication, social interaction, and perception of the world. 
- **Examples**: Using Facebook for connection, technology for daily tasks, and constant connectivity leading to distractions.

## Technological Inequality
- **Digital Divide**: Gap between those with and without access to technology.
- **Knowledge Gap**: Differences in knowledge levels based on information access.

## Evolution of Media
- **Interwoven Relationship**: Media relies on technology; both influence society.
- **Historical Impact**: The printing press revolutionized information dissemination, with tech advancements continuing to transform media.

## Types of Media and Concerns
- **Media Forms**: Comics, movies, websites, newspapers, etc.
- **Violent Media Debate**: Concerns over behaviors linked to violent games (e.g., GTA).
  - **Research Findings**: Link between violent games and aggression (Anderson and Bushman, 2001).
  - **Regulations**: ERSB rating system for games (1994); Supreme Court ruling on minors (2011).

## Global Implications of Media and Technology
- **Connecting People and Economies**: Importance of personal computing and the internet (Friedman's "The World is Flat").

## Case Studies and Real-World Impact
- **Government Censorship**: Alternatives to Western media; use of social media for government critiques.
- **Market Access**: Shared-phone programs and money transfers.
- **Cultural Imperialism Concerns**: Examined through case studies, such as mobile phones in Sub-Saharan Africa and China's internet.

## Theoretical Perspectives on Media and Technology

### Functionalism
- **Roles of Media**:
  - Commercial: Advertising.
  - Entertainment: Gaming, TV, social networking.
  - Social-Norm: Socialization tool.
  - Life-Changing: Access to digital resources.
- **Functions**: Shape perceptions and social cohesion, but may lead to dysfunctions like over-commercialization, digital addiction, and media-induced anxiety.

### Conflict Theory
- **Media Control**: Power dynamics; gatekeeping by individuals and institutions.
- **Money Influence**: Those with wealth have increased visibility in media.
- **Digital Control**: Surveillance and tracking via technology (panoptic surveillance).

### Feminist Perspective
- **Media Influence**: Beauty standards affect women's self-image and enforce gender stereotypes.
- **Cyber-Feminism**: Promotion of feminist ideals through digital platforms (e.g., #MeToo Movement, body positivity).

### Symbolic Interactionism
- **Symbolism of Technology**: Represents wealth and status; interpretations vary (e.g., Neo-Luddites vs. Technophiles).
- **Social Construction of Reality**: Media shapes our worldview and influences how we see social groups.

## Question Bank
1. **What is the digital divide?**
   - The gap between individuals with access to technology and those without.
   
2. **How has the printing press impacted information sharing?**
   - It revolutionized the dissemination of information, making it more accessible to the public.

3. **What concerns are associated with violent video games?**
   - They are linked to aggressive behavior and have prompted regulatory responses.

4. **What is gatekeeping in media?**
   - The process by which selected messages are filtered and shaped for mass media consumption.

5. **How does feminist perspective critique media?**
   - It highlights how media perpetuates beauty standards, gender stereotypes, and contributes to the gender gap in tech fields.

6. **What role does technology play in surveillance?**
   - Technology facilitates constant monitoring and tracking of individuals' movements (panoptic surveillance).

This summary encapsulates key themes and concepts from the presentation, providing a quick reference for students.